# Direct Fidelity Estimation example

In [ ]:
from pyquil.paulis import ID
from pyquil.gates import I, X, MEASURE, H, CZ, RY
from pyquil import Program, get_qc
from pyquil.api import get_benchmarker
from forest.benchmarking.direct_fidelity_estimation import ( generate_exhaustive_state_dfe_experiment, 
                                                             generate_exhaustive_process_dfe_experiment,
                                                             generate_monte_carlo_state_dfe_experiment, 
                                                             generate_monte_carlo_process_dfe_experiment,
                                                             acquire_dfe_data,
                                                             estimate_dfe )

import numpy as np
from matplotlib import pyplot

In [ ]:
# noiseless QVM
qvm = get_qc("9q-generic-qvm", as_qvm=True, noisy=False)

# noisy QVM
noisy_qvm = get_qc("9q-generic-qvm", as_qvm=True, noisy=True)

bm = get_benchmarker()

### State prep (thing we will do DFE on)

In [ ]:
p = Program()
prep_prog = p.inst(CZ(0,1))
print(prep_prog)

### Get things required for a DFE experiment ... aka experiment object

Namely return a namedtuple consisiting of 
- pauli_in, 
- prog, 
- pauli_out.

In [ ]:
# state dfe
state_exp = generate_exhaustive_state_dfe_experiment(prep_prog,[0,1],bm)

# process dfe
process_exp = generate_exhaustive_process_dfe_experiment(prep_prog,[0,1],bm)

In [ ]:
print(process_exp)

### Acquire DFE data (noiseless)

Returns dfe data 'data' and calibration data 'cal' namedtuple separately

In [ ]:
results = acquire_dfe_data(qvm, process_exp, num_shots=1000)
print("================================")
print([res.expectation for res in results])
print("================================")
print([res.calibration_expectation for res in results])

### Acquire DFE data (noisy QVM)

In [ ]:
n_results = acquire_dfe_data(noisy_qvm, process_exp, num_shots=1000)
print("================================")
print([res.expectation for res in n_results])
print("================================")
print([res.calibration_expectation for res in n_results])

### Estimate fidelity (noiseless QVM)

In [ ]:
fid_est, fid_std_err = estimate_dfe(results,'process')
print("==================================================")
print('Fidelity point estimate is',fid_est)
print('The standard error of the fidelity point estimate is', fid_std_err)
print("==================================================")

### Estimate fidelity (noisy QVM)

In [ ]:
nfid_est, nfid_std_err = estimate_dfe(n_results,'process')
print("==================================================")
print('Fidelity point estimate is', nfid_est)
print('The std error of the fidelity point estimate is', nfid_std_err)
print("==================================================")

## State fidelity between $\left|0\right\rangle$ and $R_y(\theta)\left|0\right\rangle$

In [ ]:
%%time
p = Program(I(0),I(1))
zz_state = generate_exhaustive_state_dfe_experiment(p, [0,1], bm)
zz_state_mc = generate_monte_carlo_state_dfe_experiment(p, [0,1], bm, n_terms=32)

### Exhaustive

In [ ]:
points = 10
res = np.zeros(points)
res_std_err = np.zeros(points)
for (i,theta) in enumerate(np.linspace(0, np.pi, points)): 
    zz_state.program = Program(RY(theta,0),RY(theta,1))
    zz_state_data = acquire_dfe_data(qvm,zz_state,num_shots=1000)
    fid_est, fid_std_err = estimate_dfe(zz_state_data, 'state')
    res[i] = fid_est
    res_std_err[i] = fid_std_err

In [ ]:
pyplot.errorbar(np.linspace(0, np.pi, points), res, res_std_err, fmt=".", label="Simulation")
pyplot.plot(np.linspace(0, np.pi, points), (1/2+1/2*np.cos(np.linspace(0, np.pi, points)))**2, label="Theory")
pyplot.xlabel(r"$\theta$")
pyplot.ylabel("Fidelity")
pyplot.legend()
pyplot.title(r"State fidelity between $\left|0\right\rangle$ and $R_y(\theta)\left|0\right\rangle$")

### Monte-Carlo

In [ ]:
points = 10
res = np.zeros(points)
res_std_err = np.zeros(points)
for (i,theta) in enumerate(np.linspace(0, np.pi, points)):
    zz_state_mc = generate_monte_carlo_state_dfe_experiment(p, [0,1], bm, n_terms=32)
    zz_state_mc.program = Program(RY(theta,0),RY(theta,1))
    zz_state_mc_data = acquire_dfe_data(qvm,zz_state_mc,num_shots=1000)
    fid_est, fid_std_err = estimate_dfe(zz_state_mc_data, 'state')
    res[i] = fid_est
    res_std_err[i] = fid_std_err

In [ ]:
pyplot.errorbar(np.linspace(0, np.pi, points), res, res_std_err, fmt=".", label="Simulation")
pyplot.plot(np.linspace(0, np.pi, points), (1/2+1/2*np.cos(np.linspace(0, np.pi, points)))**2, label="Theory")
pyplot.xlabel(r"$\theta$")
pyplot.ylabel("Fidelity")
pyplot.legend()
pyplot.title(r"State fidelity between $\left|0\right\rangle$ and $R_y(\theta)\left|0\right\rangle$")

## State fidelity between $\left|0\right\rangle\left|0\right\rangle$ and $R_y(\theta)\otimes R_y(2\theta)\left|0\right\rangle\left|0\right\rangle$

### Exhaustive

In [ ]:
points = 10
res = np.zeros(points)
res_std_err = np.zeros(points)
for (i,theta) in enumerate(np.linspace(0, np.pi, points)): 
    zz_state.program = Program(RY(theta,0),RY(2*theta,1))
    zz_state_data = acquire_dfe_data(qvm,zz_state,num_shots=1_000)
    fid_est, fid_std_err = estimate_dfe(zz_state_data, 'state')
    res[i] = fid_est
    res_std_err[i] = fid_std_err

In [ ]:
pyplot.errorbar(np.linspace(0, np.pi, points), res, res_std_err, fmt=".", label="simulation")
pyplot.plot(np.linspace(0, np.pi, points), 
            (1/2+1/2*np.cos(np.linspace(0, np.pi, points)))*(1/2+1/2*np.cos(2*np.linspace(0, np.pi, points))),
            label="theory")
pyplot.xlabel(r"$\theta$")
pyplot.ylabel("Fidelity")
pyplot.legend()
pyplot.title(r"State fidelity between $\left|0\right\rangle\left|0\right\rangle$ and $R_y(\theta)\otimes R_y(2\theta)\left|0\right\rangle\left|0\right\rangle$");

### Monte Carlo

In [ ]:
points = 10
res = np.zeros(points)
res_std_err = np.zeros(points)
for (i,theta) in enumerate(np.linspace(0, np.pi, points)): 
    zz_state_mc = generate_monte_carlo_state_dfe_experiment(p, [0,1], bm, n_terms=32)
    zz_state_mc.program = Program(RY(theta,0),RY(2*theta,1))
    zz_state_mc_data = acquire_dfe_data(qvm,zz_state_mc,num_shots=1_000)
    est = estimate_dfe(zz_state_mc_data, 'state')
    res[i] = fid_est
    res_std_err[i] = fid_std_err

In [ ]:
pyplot.errorbar(np.linspace(0, np.pi, points), res, res_std_err, fmt=".", label="simulation")
pyplot.plot(np.linspace(0, np.pi, points), 
            (1/2+1/2*np.cos(np.linspace(0, np.pi, points)))*(1/2+1/2*np.cos(2*np.linspace(0, np.pi, points))),
            label="theory")
pyplot.xlabel(r"$\theta$")
pyplot.ylabel("Fidelity")
pyplot.legend()
pyplot.title(r"State fidelity between $\left|0\right\rangle\left|0\right\rangle$ and $R_y(\theta)\otimes R_y(2\theta)\left|0\right\rangle\left|0\right\rangle$")

## Process fidelity between $I\otimes I$ and $R_y(\theta)\otimes I$

In [ ]:
%%time
p = Program(I(0))
ii_proc = generate_exhaustive_process_dfe_experiment(p, [0], bm)
ii_proc_mc = generate_monte_carlo_process_dfe_experiment(p, [0], bm, n_terms=32)

### Exhaustive

In [ ]:
points = 10
res = np.zeros(points)
res_std_err = np.zeros(points)
for (i,theta) in enumerate(np.linspace(0, np.pi, points)): 
    ii_proc.program = Program(RY(theta,0))
    ii_proc_data = acquire_dfe_data(qvm,ii_proc,num_shots=500)
    fid_est, fid_std_err = estimate_dfe(ii_proc_data, 'process')
    res[i] = fid_est
    res_std_err[i] = fid_std_err

In [ ]:
pyplot.errorbar(np.linspace(0, np.pi, points), res, res_std_err, fmt=".", label="data")
pyplot.plot(np.linspace(0, np.pi, points), 
            (4*np.cos(np.linspace(0, np.pi, points)/2)**2+2)/6,
            label="theory")
#pyplot.axhline(0.2380952380952381+0.1, color="red", ls="--")
pyplot.legend()
pyplot.ylim(0,1.25)
pyplot.title(r"Process fidelity between $I\otimes I$ and $R_y(\theta)\otimes I$")

### Monte Carlo

In [ ]:
points = 10
res = np.zeros(points)
res_std_err = np.zeros(points)
for (i,theta) in enumerate(np.linspace(0, np.pi, points)): 
    ii_proc_mc = generate_monte_carlo_process_dfe_experiment(p, [0], bm, n_terms=32)
    ii_proc_mc.program = Program(RY(theta,0))
    ii_proc_mc_data = acquire_dfe_data(qvm, ii_proc_mc, num_shots=500)
    fid_est, fid_std_err = estimate_dfe(ii_proc_mc_data, 'process')
    res[i] = fid_est
    res_std_err[i] = fid_std_err

In [ ]:
pyplot.errorbar(np.linspace(0, np.pi, points), res, res_std_err, fmt=".", label="data")
pyplot.plot(np.linspace(0, np.pi, points), 
            (4*np.cos(np.linspace(0, np.pi, points)/2)**2+2)/6,
            label="theory")
pyplot.legend()
pyplot.ylim(0,1.25)
pyplot.title(r"Process fidelity between $I\otimes I$ and $R_y(\theta)\otimes I$")